In [71]:
import os

In [103]:
%pwd

'c:\\Users\\user\\Desktop\\wine quality\\wine-quality-predictor'

In [73]:
os.chdir("Desktop\wine quality\wine-quality-predictor")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Desktop\\wine quality\\wine-quality-predictor'

In [ ]:
%pwd

'c:\\Users\\user\\Desktop\\wine quality\\wine-quality-predictor'

In [115]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path 

In [116]:
from wine_quality_prediction.constants.init_ import *
from wine_quality_prediction.utils.common import read_yaml, create_directories

In [117]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
          root_dir = config.root_dir,
          source_URL = config.source_URL,
          local_data_file = config.local_data_file,
          unzip_dir = config. unzip_dir
          )

          return data_ingestion_config

In [118]:
import os
import urllib.request as request
import zipfile
from wine_quality_prediction import logger
from wine_quality_prediction.utils.common import get_size

In [119]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [120]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-17 19:14:18,598: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-17 19:14:18,725: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-17 19:14:18,734: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-17 19:14:18,761: INFO: common: created directory at : artifacts]
[2025-02-17 19:14:18,770: INFO: common: created directory at : artifacts/data_ingestion]
[2025-02-17 19:14:20,599: INFO: 4121774386: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 24192
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d82780d41497d72e03c28ca2092ae8d835b47242243aeb07c60930bab42eca33"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E00E:2ED9B8:2A80B7:3F635E:67B33D33
Accept-Ranges: bytes
Date: Mon, 